## Desafío 1 - Procesamiento del Lenguaje Natural I
##### Docentes: Rodrigo Cárdenas / Nicolás  Vattuone
##### Autora: María Luz Micozzi

### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [3]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [4]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()

In [5]:
# en el atributo `data` accedemos al texto
print(newsgroups_train.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [6]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [7]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [8]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [9]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [10]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [11]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [12]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [13]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [14]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [15]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([4811, 6635, 4253, ..., 9019, 9016, 8748])

In [16]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [17]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [18]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [19]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [20]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [21]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


In [22]:
# imports
import random
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report

### 1. Vectorizar documetos

In [23]:
random.seed(42)

# Elegimos 5 índices aleatorios
random_indexes = random.sample(range(len(newsgroups_train.data)), 5)

random_indexes

[10476, 1824, 409, 4506, 4012]

In [24]:
# Función para mostrars índice, categoría y parte del documento
def show_document(i=0):
    print(f"Índice: {i}")
    categoria = newsgroups_train.target_names[newsgroups_train.target[i]]
    print(f"Categoría: {categoria}")
    print(" ")
    print(newsgroups_train.data[i][:2000])

# Función para mostrar documentos similares
def show_most_similar_documents(i=0, n=5):
  # Medimos la similaridad coseno con todos los documentos de train
  cossim = cosine_similarity(X_train[i], X_train)[0]

  # Obtenemos los 5 documentos más similares:
  mostsim = np.argsort(cossim)[::-1][1:6]

  # Mostramos los 5 documentoss más similares
  for idx in mostsim:
    show_document(idx)
    print("-" * 100)

#### Primer documento

In [25]:
index = random_indexes[0]
# Mostramos el documento
show_document(i=index)

Índice: 10476
Categoría: rec.sport.hockey
 
This is a general question for US readers:

How extensive is the playoff coverage down there?  In Canada, it is almost
impossible not to watch a series on TV (ie the only two series I have not had
an opportunity to watch this year are Wash-NYI and Chi-Stl, the latter because
I'm in the wrong time zone!).  We (in Canada) are basically swamped with 
coverage, and I wonder how many series/games are televised nationally or even
locally in the US and how much precedence they take over, say, local news if
the games go into double-OT.

Email me so as not to waste bandwidth, please.  My news feed is kind of slow
anyways.


In [26]:
# Mostramos los 5 documentos más similares
show_most_similar_documents(i=index, n=5)

Índice: 5064
Categoría: rec.sport.hockey
 

I only have one comment on this:  You call this a *classic* playoff year
and yet you don't include a Chicago-Detroit series.  C'mon, I'm a Boston
fan and I even realize that Chicago-Detroit games are THE most exciting
games to watch.
----------------------------------------------------------------------------------------------------
Índice: 9623
Categoría: talk.politics.mideast
 
Accounts of Anti-Armenian Human Right Violations in Azerbaijan #012
                 Prelude to Current Events in Nagorno-Karabakh

        +---------------------------------------------------------+
        |                                                         |
        |  I saw a naked girl with her hair down. They were       |
        |  dragging her. She kept falling because they were       |
        |  pushing her and kicking her. She fell down, it was     |
        |  muddy there, and later other witnesses who saw it from |
        |  their balconies told u

***Análisis:***

- **Índice: 5064 - Categoría: rec.sport.hockey**

  El documento tiene la misma categoría que el original, además podemos observar que comparten palabras como playoff, games y series.
---
- **Índice: 9623 - Categoría: talk.politics.mideast**
- **Índice: 10575 - Categoría: sci.crypt**
- **Índice: 10836 - Categoría: alt.atheism**
- **Índice: 2350 - Categoría: sci.crypt**

  Los documentos tienen categorías muy diferentes al documento original, y a simple vista no se observan términos significativos que sean similares. Tal vez hay alguna similitud en la redacción / tono o compartan palabras de uso general.





#### Segundo documento

In [27]:
index = random_indexes[1]
# Mostramos el documento
show_document(i=index)

Índice: 1824
Categoría: comp.sys.mac.hardware
 


	I think this kind of comparison is pretty useless in general.  The
processor is only good when a good computer is designed around it adn the
computer is used in its designed purpose.  Comparing processor speed is
pretty dumb because all you have to do is just increase the clock speed
to increase speed among other things.

	I mean how can you say a 040 is faster than a 486 without 
giving is operational conditions?  Can you say the same when 
you are running a program that uses a lot of transidental functions.
Knowing that 040 does not have transidental functions building in to 
its FPU and 486 does, can you say that 040 is still faster?

	Anyway, I hope people do not decided upon wether a computers
is good or not solely on its processor.  Or how fast a processor is
based on its name, because one can alway do a certain things to a
processor to speed it up.  

	But if we restrict our arguements to, for example, pure
processor architectur

In [28]:
# Mostramos los 5 documentos más similares
show_most_similar_documents(i=index, n=5)

Índice: 9921
Categoría: comp.sys.mac.hardware
 
dhk@ubbpc.uucp (Dave Kitabjian) writes ...

040 486 030 386 020 286


060 fastest, then Pentium, with the first versions of the PowerPC
somewhere in the vicinity.


No.  Computer speed is only partly dependent of processor/clock speed.
Memory system speed play a large role as does video system speed and
I/O speed.  As processor clock rates go up, the speed of the memory
system becomes the greatest factor in the overall system speed.  If
you have a 50MHz processor, it can be reading another word from memory
every 20ns.  Sure, you can put all 20ns memory in your computer, but
it will cost 10 times as much as the slower 80ns SIMMs.

And roughly, the 68040 is twice as fast at a given clock
speed as is the 68030.

----------------------------------------------------------------------------------------------------
Índice: 6364
Categoría: comp.sys.mac.hardware
 
Well folks, after some thought the answer struck me flat in the face:

"Why would Ap

***Análisis:***

- **Índice: 9921 - Categoría: comp.sys.mac.hardware**
- **Índice: 6364 - Categoría: comp.sys.mac.hardware**
- **Índice: 5509 - Categoría: comp.sys.mac.hardware**
- **Índice: 2614 - Categoría: comp.sys.mac.hardware**
- **Índice: 4359 - Categoría: comp.sys.mac.hardware**

  Todos los documentos tienen la misma categoría que el original, además de compartir muchas palabras iguales o relacionadas, como computer, memory, entre otras.

#### Tercer documento

In [29]:
index = random_indexes[2]
# Mostramos el documento
show_document(i=index)

Índice: 409
Categoría: comp.graphics
 
I can't fiqure this out.  I have properly compiled pov on a unix machine
running SunOS 4.1.3  The problem is that when I run the sample .pov files and
use the EXACT same parameters when compiling different .tga outputs.  Some
of the .tga's are okay, and other's are unrecognizable by any software.


In [30]:
# Mostramos los 5 documentos más similares
show_most_similar_documents(i=index, n=5)

Índice: 3444
Categoría: comp.graphics
 
Hi, I'm just getting into PoVRay and I was wondering if there is a graphic
package that outputs .POV files.  Any help would be appreciated.
Thanks.

Later'ish
Craig

----------------------------------------------------------------------------------------------------
Índice: 5799
Categoría: comp.graphics
 
I finally got a 24 bit viewer for my POVRAY generated .TGA files.

It was written in C by Sean Malloy and he kindly sent me a copy.  He
wrote it for the same purpose, to view .TGA files using his SpeedStar 24.

It ONLY works with the SpeedStar 24 and I cannot send copies since it is
not my program.  I believe the author may release a version at a future
time when the program is more developed.   He may or may not comment on
this, as he pleases.

Thanks to all who were helpful.

Regards,
----------------------------------------------------------------------------------------------------
Índice: 5905
Categoría: comp.graphics
 

Hallo POV-Renderers

***Análisis:***

- **Índice: 3444 - Categoría: comp.graphics**
- **Índice: 5799 - Categoría: comp.graphics**
- **Índice: 5905 - Categoría: comp.graphics**
- **Índice: 1764 - Categoría: comp.graphics**
- **Índice: 3364 - Categoría: comp.graphics**

  Todos los documentos tienen la misma categoría que el original, además de compartir muchas palabras iguales o relacionadas, como pov, files, entre otras.

#### Cuarto documento

In [31]:
index = random_indexes[3]
# Mostramos el segundo documento
show_document(i=index)

Índice: 4506
Categoría: rec.autos
 

This does sound good, but I heard it tends to leave more grit, etc in the 
oil pan.  Also, I've been told to change the old when it's hot before the
grit has much time to settle.

Any opinions?



In [32]:
# Mostramos los 5 documentos similares
show_most_similar_documents(i=index, n=5)

Índice: 4211
Categoría: rec.motorcycles
 


It's normal for the BMW K bikes to use a little oil in the first few thousand 
miles.  I don't know why.  I've had three new K bikes, and all three used a
bit of oil when new - max maybe .4 quart in first 1000 miles; this soon quits
and by the time I had 10,000 miles on them the oil consumption was about zero.
I've been told that the harder you run the bike (within reason) the sooner
it stops using any oil.

----------------------------------------------------------------------------------------------------
Índice: 5928
Categoría: comp.sys.mac.hardware
 
or
there


Okay, I guess its time for a quick explanation of Mac sound.

The original documentation for the sound hardware (IM-3) documents how to
make sound by directly accessing hardware.  Basically, you jam values
into all the even bytes from SoundBase to SoundBase+0x170. This was
because
of how the Mac 128 (and some later machines) generated sound was by
scanning
this block and D/Aing eve

***Análisis:***

- **Índice: 4211 - Categoría: rec.motorcycles**

  El documento tiene una categoría muy similar y relacionada al original, además de compartir palabras iguales o relacionadas, como oil.
---
- **Índice: 5928 - Categoría: comp.sys.mac.hardware**

  EL documento tiene categoría muy diferente al documento original, y a simple vista no se observan términos significativos que sean similares. Tal vez hay alguna similitud en la redacción / tono o compartan palabras de uso general.
---
- **Índice: 6224 - Categoría: rec.autos**
- **Índice: 5171 - Categoría: rec.autos**
- **Índice: 9491 - Categoría: rec.autos**

  Los documentos tienen la misma categoría que el original, además de compartir palabras iguales o relacionadas, como oil.

#### Quinto documento

In [33]:
index = random_indexes[4]
# Mostramos el segundo documento
show_document(i=index)

Índice: 4012
Categoría: rec.sport.hockey
 
For those Leaf fans who are concerned, the following players are slated for
return on Thursday's Winnipeg-Toronto game :
    Peter Zezel, John Cullen

  Mark Osborne and Dave Ellett are questionable to return on Thursday.


In [34]:
# Mostramos los 5 documentos similares
show_most_similar_documents(i=index, n=5)

Índice: 6599
Categoría: soc.religion.christian
 
True.

Also read 2 Peter 3:16

Peter warns that the scriptures are often hard to understand by those who
are not learned on the subject.
----------------------------------------------------------------------------------------------------
Índice: 10644
Categoría: rec.sport.hockey
 
In  <1qvos8$r78@cl.msu.>, vergolin@euler.lbs.msu.edu (David Vergolini) writes...

There's quite a few Wings fans lurking about here, they just tend
to be low key and thoughtful rather than woofers.  I suppose every
family must have a Roger Clinton, though.  But remember (to paraphrase
one of my favorite Star Trek lines), "if we adopt the ways of the Leaf
fans, we are as bad as the Leaf fans".

Ron
----------------------------------------------------------------------------------------------------
Índice: 7478
Categoría: rec.sport.hockey
 
Toronto                          1 1 1--3
Detroit                          1 4 1--6
First period
     1, Detroit, Yzerman 1 

***Análisis:***

- **Índice: 6599 - Categoría: soc.religion.christian**

  EL documento tiene categoría muy diferente al documento original, y a simple vista no se observan términos significativos que sean similares. Tal vez hay alguna similitud en la redacción / tono o compartan palabras de uso general.
---
- **Índice: 10644 - Categoría: rec.sport.hockey**
- **Índice: 7478 - Categoría: rec.sport.hockey**
- **Índice: 7308 - Categoría: rec.sport.hockey**

  Los documentos tienen la misma categoría que el original, además de compartir palabras iguales o relacionadas, como Toronto, players, game, fans.
---
- **Índice: 10792 - Categoría: rec.sport.baseball**

  El documento tiene una categoría muy similar y relacionada al original, además de compartir palabras iguales o relacionadas, como Toronto, players, game.

Salvo en algunos casos particulares como por ejemplo el primer documento, podemos ver que en general los textos identificados como similares tienen la misma categoría o una categoría similar, y además comparten vocabulario y son parecidos en su redacción.

Es destacable también que si bien el primer y último documento tienen la misma categoría (rec.sport.hockey), para el primero se ve un resultado bastante más pobre a la hora de estudiar los documentos similares.

### 2. Entrenar modelos de clasificación Naïve Bayes

Se realizará una búsqueda aleatoria de hiperparámetros utilizando RandomizedSearchCV. El modelo se basa en un pipeline que incluye un TfidfVectorizer para la representación de texto y un clasificador Naive Bayes (MultinomialNB o ComplementNB).

Parámetros del vectorizador:

- max_df: valores entre 0.5 y 1.0,
- min_df: desde 1 hasta 5,
- ngram_range: unigramas y bigramas,
- stop_words: con y sin eliminación de palabras vacías en inglés,
- sublinear_tf: con y sin escalado logarítmico.

También se exploran distintos valores para el parámetro alpha del clasificador (0.001, 0.01, 0.1, 0.5, 1.0).

La búsqueda se hará sobre 50 combinaciones aleatorias, usando validación cruzada de 5 pliegues. La métrica utilizada para evaluar el rendimiento de cada configuración será el F1-score macro.


In [35]:
# Crear un pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB())
])

# Definir el espacio de búsqueda
param_distributions = {
    'tfidf__max_df': np.linspace(0.5, 1.0, 5),
    'tfidf__min_df': [1, 2, 3, 5],
    'tfidf__ngram_range': [(1,1), (1,2)],
    'tfidf__stop_words': ['english', None],
    'tfidf__sublinear_tf': [True, False],
    'clf': [MultinomialNB(), ComplementNB()],
    'clf__alpha': [0.001, 0.01, 0.1, 0.5, 1.0]
}

# Randomized search
search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=50,
    scoring='f1_macro',
    cv=5,
    verbose=1,
    n_jobs=-1,
    random_state=42
)

In [36]:
# Entrenamos
y_train = newsgroups_train.target
y_test = newsgroups_test.target
search.fit(newsgroups_train.data, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                             ('clf', MultinomialNB())]),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'clf': [MultinomialNB(),
                                                ComplementNB()],
                                        'clf__alpha': [0.001, 0.01, 0.1, 0.5,
                                                       1.0],
                                        'tfidf__max_df': array([0.5  , 0.625, 0.75 , 0.875, 1.   ]),
                                        'tfidf__min_df': [1, 2, 3, 5],
                                        'tfidf__ngram_range': [(1, 1), (1, 2)],
                                        'tfidf__stop_words': ['english', None],
                                        'tfidf__sublinear_tf': [True, False]},
                   random_state=42, scoring='f1_macro', verbose=1)

In [37]:
# Vemos el top 5 de mejores párametros encontrados
pd.set_option('display.max_colwidth', None)

results_df = pd.DataFrame(search.cv_results_)
top_results = results_df.sort_values(by='mean_test_score', ascending=False).head(5)

print("\nTop 5 combinaciones de hiperparámetros:")
print(top_results[['mean_test_score', 'params']])


Top 5 combinaciones de hiperparámetros:
    mean_test_score  \
6          0.778489   
49         0.764499   
22         0.760426   
31         0.759381   
19         0.758518   

                                                                                                                                                                                params  
6     {'tfidf__sublinear_tf': False, 'tfidf__stop_words': 'english', 'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 1, 'tfidf__max_df': 1.0, 'clf__alpha': 0.1, 'clf': ComplementNB()}  
49        {'tfidf__sublinear_tf': False, 'tfidf__stop_words': None, 'tfidf__ngram_range': (1, 1), 'tfidf__min_df': 1, 'tfidf__max_df': 0.75, 'clf__alpha': 0.1, 'clf': ComplementNB()}  
22   {'tfidf__sublinear_tf': True, 'tfidf__stop_words': 'english', 'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 2, 'tfidf__max_df': 0.875, 'clf__alpha': 1.0, 'clf': ComplementNB()}  
31  {'tfidf__sublinear_tf': False, 'tfidf__stop_words': 'english', 'tfidf__ngram

Luego de realizar la búsqueda se obtuvo la mejor combinación de parámetros en la iteración número 6. Esta configuración alcanzó un puntaje promedio de F1 macro de aproximadamente 0.7784 en el entrenamiento.

Los parámetros óptimos encontrados fueron los siguientes:

- sublinear_tf: False
- stop_words: english
- ngram_range: (1, 2)
- min_df: 1
- max_df: 1.0
- clasificador: ComplementNB()
- alpha: 0.1

In [38]:
# Evaluamos el mejor modelo
best_model = search.best_estimator_
y_pred = best_model.predict(newsgroups_test.data)

# F1-score
f1_best_model = f1_score(y_test, y_pred, average='macro')
print(f"F1-score (macro): {f1_best_model:.4f}")

# Classification report
print("Reporte de clasificación:")
print(classification_report(y_test, y_pred, target_names=newsgroups_test.target_names))

F1-score (macro): 0.7098
Reporte de clasificación:
                          precision    recall  f1-score   support

             alt.atheism       0.34      0.48      0.39       319
           comp.graphics       0.73      0.72      0.73       389
 comp.os.ms-windows.misc       0.69      0.62      0.65       394
comp.sys.ibm.pc.hardware       0.66      0.69      0.67       392
   comp.sys.mac.hardware       0.79      0.71      0.75       385
          comp.windows.x       0.80      0.80      0.80       395
            misc.forsale       0.78      0.79      0.78       390
               rec.autos       0.81      0.74      0.77       396
         rec.motorcycles       0.82      0.77      0.79       398
      rec.sport.baseball       0.93      0.84      0.88       397
        rec.sport.hockey       0.88      0.94      0.91       399
               sci.crypt       0.77      0.81      0.79       396
         sci.electronics       0.73      0.58      0.64       393
                 sci.med

Se logró una mejora significativa al pasar del modelo original (MultinomialNB sin ajuste de hiperparámetros con F1 macro de **0.5854**) a un modelo ComplementNB con búsqueda aleatoria de hiperparámetros, alcanzando un F1 macro de **0.7098**.

El reporte de clasificación muestra un rendimiento especialmente alto en categorías como 'rec.sport.hockey' y 'rec.sport.baseball', mientras que las clases con peores resultados fueron 'alt.atheism' y 'talk.religion.misc'.


### 3. Transponer la matriz documento-término

Se tomaron las siguientes palabras de los documentos elegidos al azar del punto 1: **canada** - **hope** - **oil** - **software** - **player**

In [40]:
# Transponemos la matriz
X_T = X_train.T

words = ['canada', 'hope', 'oil', 'software', 'player']

word_indices = [tfidfvect.vocabulary_.get(word) for word in words if word in tfidfvect.vocabulary_]

# Aplicamos cosine similarity a las palabras seleccionadas para buscar las palabras similares
word_similarity = cosine_similarity(X_T[word_indices], X_T)

# Mostramos para cada palabra las 5 más similares
for i, word in enumerate(words):
    if word in tfidfvect.vocabulary_:
        similarity_scores = word_similarity[i]
        most_similar_indices = np.argsort(similarity_scores)[::-1][1:6]  # excluye la palabra consigo misma
        most_similar_words = [(tfidfvect.get_feature_names_out()[idx], similarity_scores[idx])
                              for idx in most_similar_indices]

        print(f"\nPalabras con mayor similitud a '{word}':")
        for similar_word, score in most_similar_words:
            print(f"  {similar_word:<15} Similitud: {score:.4f}")


Palabras con mayor similitud a 'canada':
  olympiahalle    Similitud: 0.2643
  xs1100          Similitud: 0.2062
  censured        Similitud: 0.2012
  sweden          Similitud: 0.1959
  uncirculated    Similitud: 0.1955

Palabras con mayor similitud a 'hope':
  helps           Similitud: 0.3336
  nobles          Similitud: 0.2003
  to              Similitud: 0.1505
  binyamin        Similitud: 0.1442
  ellected        Similitud: 0.1442

Palabras con mayor similitud a 'oil':
  presurized      Similitud: 0.2701
  volitiles       Similitud: 0.2701
  prolonging      Similitud: 0.2674
  decant          Similitud: 0.2663
  quart           Similitud: 0.2609

Palabras con mayor similitud a 'software':
  maturity        Similitud: 0.2296
  humphrey        Similitud: 0.2216
  repeatable      Similitud: 0.2040
  hardware        Similitud: 0.1676
  standdown       Similitud: 0.1667

Palabras con mayor similitud a 'player':
  grounder        Similitud: 0.2802
  sac             Similitud: 0.2360
 

***Análisis***

- **canada**  
  olympiahalle y sweden indican asociaciones geográficas o culturales, posiblemente relacionadas con eventos o localizaciones en países con tradición deportiva o política. XS1100 puede referirse al modelo de motocicleta mencionado en la categoría rec.motorcycles, aunque no parece haber una relación semátinca directa con canada

- **hope**  
  helps y nobles reflejan vínculos con el apoyo, la generosidad y el impulso positivo, sugiriendo un contexto de asistencia o valores elevados.

- **oil**  
  pressurized y volitiles apuntan a características físicas o químicas propias del aceite, como presión y volatilidad, dentro de un marco industrial o científico. Quart puede referirse a una medida de aceite.

- **software**  
  maturity y hardware conectan el término con etapas de desarrollo tecnológico y su relación con componentes físicos, en un entorno de ingeniería o producto. Humphrey podría referirse a Watts Humphrey, reconodido en el mundo del software.

- **player**  
  grounder, sac, team y player parecen estar asociadas a lo deportivo, en particular al baseball, una de las categorías de los documentos del dataset.

En conclusión, si bien las medidas de similitud encontradas son bajas, en general es posible encontrar una relación entre las palabras dentro del contexto de los textos analizados.